In [1]:
import numpy as np
import pandas as pd
import os
import json

cwd = os.getcwd()

results_path = cwd[:-28] + "Results/T5_wiki_basic_ft_test/all_scores_range_0_137_trail_{}.npz"
results_path_sbert = cwd[:-28] + "Results/T5_wiki_basic_ft_test/all_sbert_scores_0_137_trail_{}.npz"

results_intent_path = cwd[:-28] + "Results/T5_wiki_basic_ft_test/all_scores_range_by_intent_0_137_trail_{}.txt"

intents = ["What drives the economy in this state?",
           "What are the available modes of transport in this state?",
           "How is the government structured in this state?",
           "What about this state's arts and culture attracts you the most?",
           "The main reasons why you would like living in this state"]

sample_size = 2

model = "T5 Wiki-Transfer Basic-FT"

In [2]:

total_data = None
total_data_sbert = []
total_data_intent = {}

for intent in intents:
    total_data_intent[intent] = []

# get all of the "normal" data
for i in range(sample_size):
    temp = np.load(results_path.format(i))
    sbert_temp = np.load(results_path_sbert.format(i))
    data = temp['scores']
    data_sbert = sbert_temp['scores']
    
    if total_data is None:
        # we need to take the mean of axis-one first because each "user-sample" contains a set of predicted summaries. So we first avg those sets, before average across all the data/trials
        total_data = data.mean(axis=1)
        total_data_sbert = data_sbert.mean(axis=1)
    else:
        total_data = np.concatenate((total_data, data.mean(axis=1)), axis=0)
        total_data_sbert = np.concatenate((total_data_sbert, data_sbert.mean(axis=1)), axis=0)

# get all of the "by-intent" data
for i in range(sample_size):
    res = None
    with open(results_intent_path.format(i)) as f:
        res = json.load(f)

    for intent in intents:
        total_data_intent[intent].extend(np.mean(res[intent], axis=1).tolist())

# now calculate the relavent stats
total_mean = np.mean(total_data, axis=0)
total_stddev = np.std(total_data, axis=0)

total_mean_sbert = np.mean(total_data_sbert, axis=0)
total_stddev_sbert = np.std(total_data_sbert, axis=0)

mean_intent = {}
stddev_intent = {}

for intent in intents:
    mean = np.mean(total_data_intent[intent], axis=0)
    stddev = np.std(total_data_intent[intent], axis=0)
    mean_intent[intent] = mean
    stddev_intent[intent] = stddev

metrics = ["Rouge-1", "Rouge-2", "Rouge-L"]

for i, (row_m, row_s) in enumerate(zip(total_mean, total_stddev)):
    row_str = model + " & "
    for col_m, col_s in zip(row_m, row_s):
        row_str += str(np.round(col_m, 4)) + " $\\pm$ " + str(np.round(col_s, 4)) + " & "
    
    row_str = row_str[:-3]
    row_str += " \\\\"
    print(metrics[i])
    print(row_str)
print("SBERT Cosime Sim. Score: ")
print(model + " & " + str(np.round(total_mean_sbert, 4)) + " $\\pm$ " + str(np.round(total_stddev_sbert, 4)) + " \\\\")

print("\n\n\n")

Rouge-1
T5 Wiki-Transfer Basic-FT & 0.137 $\pm$ 0.0367 & 0.5422 $\pm$ 0.0845 & 0.2102 $\pm$ 0.0429 & 0.1314 $\pm$ 0.0268 \\
Rouge-2
T5 Wiki-Transfer Basic-FT & 0.0359 $\pm$ 0.0149 & 0.1551 $\pm$ 0.0765 & 0.0561 $\pm$ 0.023 & 0.0351 $\pm$ 0.0144 \\
Rouge-L
T5 Wiki-Transfer Basic-FT & 0.0945 $\pm$ 0.0232 & 0.3823 $\pm$ 0.0678 & 0.1456 $\pm$ 0.0268 & 0.091 $\pm$ 0.0167 \\
SBERT Cosime Sim. Score: 
T5 Wiki-Transfer Basic-FT & 0.4841 $\pm$ 0.0753 \\






In [3]:
metrics = ["Rouge-1", "Rouge-2", "Rouge-L"]

for intent in intents:
    print("\nResults for intent {}\n".format(intent))
    for i, (row_m, row_s) in enumerate(zip(mean_intent[intent], stddev_intent[intent])):
        row_str = "\t"+model + " & "
        for col_m, col_s in zip(row_m, row_s):
            row_str += str(np.round(col_m, 4)) + " $\\pm$ " + str(np.round(col_s, 4)) + " & "
        
        row_str = row_str[:-3]
        row_str += " \\\\"
        print("\t"+metrics[i])
        print(row_str)


Results for intent What drives the economy in this state?

	Rouge-1
	T5 Wiki-Transfer Basic-FT & 0.1396 $\pm$ 0.0348 & 0.5343 $\pm$ 0.0704 & 0.2135 $\pm$ 0.0409 & 0.1335 $\pm$ 0.0255 \\
	Rouge-2
	T5 Wiki-Transfer Basic-FT & 0.0375 $\pm$ 0.0154 & 0.1525 $\pm$ 0.0567 & 0.0582 $\pm$ 0.0222 & 0.0363 $\pm$ 0.0139 \\
	Rouge-L
	T5 Wiki-Transfer Basic-FT & 0.096 $\pm$ 0.0225 & 0.3751 $\pm$ 0.0564 & 0.1474 $\pm$ 0.0263 & 0.0921 $\pm$ 0.0164 \\

Results for intent What are the available modes of transport in this state?

	Rouge-1
	T5 Wiki-Transfer Basic-FT & 0.1374 $\pm$ 0.0326 & 0.5195 $\pm$ 0.0729 & 0.2086 $\pm$ 0.0364 & 0.1303 $\pm$ 0.0228 \\
	Rouge-2
	T5 Wiki-Transfer Basic-FT & 0.0301 $\pm$ 0.0075 & 0.1213 $\pm$ 0.0388 & 0.0463 $\pm$ 0.0098 & 0.0289 $\pm$ 0.0061 \\
	Rouge-L
	T5 Wiki-Transfer Basic-FT & 0.0926 $\pm$ 0.0201 & 0.3569 $\pm$ 0.0577 & 0.1412 $\pm$ 0.0222 & 0.0882 $\pm$ 0.0139 \\

Results for intent How is the government structured in this state?

	Rouge-1
	T5 Wiki-Transfer Basic